# **import tất cả thư viện cần thiết**

In [ ]:
# Thêm thư viện
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from keras.applications import VGG16
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from keras.applications import VGG16
from keras.layers import Input
from keras.models import Model
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.layers.core import Flatten
import numpy as np
import random
import os

Using TensorFlow backend.


# Truy cập vào drive để lấy dữ liệu

****




In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/Colab\ Notebooks/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


****

# Truy cập vào forder chứa data sau đó lưu lại đường dẫn kèm theo nhãn là tên forder chứa ảnh

In [ ]:
#Lấy tất cả đường đẫn đến ảnh 
filenames=os.listdir("data/")
print(filenames)
labels=[]
paths=[]
images=[]
c=0
for i in filenames:
  path=os.path.join("data/",i)
  file_images=os.listdir(path)
  for j,f in enumerate(file_images):
    image=os.path.join(path,f)
    images.append(image)
#Xáo trộn đường dẫn
random.shuffle(images)
images

****




# Label data ở dạng One-hot Encoding

In [ ]:
from keras.utils import np_utils 
labels = [int(p.split(os.path.sep)[-2]) for p in images]
Labels = np_utils.to_categorical(labels, 35) 
Labels

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

# **Đọc ảnh và tiền xử lý ảnh**





> Resize ảnh về kích thước chung là 28*28


> Chuyển ảnh sang ảnh xám 


> Làm mờ ảnh để khử nhiễu sau đó chuyển ảnh về dạng Binary









In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications import imagenet_utils
import cv2
Images = []
for (j, imagePath) in enumerate(images):
  img = cv2.imread(imagePath);
  img=cv2.resize(img, (28,28))
  roi_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
  roi_blur = cv2.GaussianBlur(roi_gray,(3,3),1)
  ret, thre = cv2.threshold(roi_blur,0,255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
  Images.append(thre)
Images = np.array(Images)
print(Images.shape)

(1714, 28, 28)


# **Chia tập dữ liệu**




> Chia tập dữ liệu 90% cho training và 10% cho test



In [ ]:
Images = Images.reshape(Images.shape[0], 28, 28, 1) 
X_train, X_test, y_train, y_test = train_test_split(Images, Labels, test_size=0.1, random_state=42)

# Xây dựng model

****


> Sử dụng 5 layers, hàm loss sử dụng 'categorical_crossentropy', optimizer sử dụng 'adam'



In [ ]:
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Activation, Flatten 
from keras.layers import Conv2D, MaxPooling2D 
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28,28,1)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(35, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 4608)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               589952    
_________________________________________________________________
dense_6 (Dense)              (None, 35)                4515      
Total params: 604,035
Trainable params: 604,035
Non-trainable params: 0
________________________________________________

# Traning

**Bắt đầu traning**


> Traning với batch size = 32, epochs = 30, nomalization dữ liệu trước khi traning



In [ ]:
His = model.fit(X_train/255.0, y_train, validation_data=(X_test/255.0, y_test), batch_size=32, epochs=30, verbose=1)

# Kết quả

**Kết quả thu được**

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0) 
print("epochs :",30)
print("score :", score[1])
print("[+] Finished")

epochs : 30
score : 0.9242
[+] Finished


**Lưu model**

In [ ]:
model.save("model_plate/model.h5")